In [25]:
import matplotlib
import numpy as np
import math
import itertools
import sklearn
import pydotplus
import warnings
import mysql.connector as myconnect
import pandas as pd
import sklearn.linear_model
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from sklearn.linear_model import LinearRegression  
from sklearn.model_selection import train_test_split, KFold, cross_val_score, LeaveOneOut
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn import tree
from scipy import stats
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.kernel_ridge import KernelRidge
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.gaussian_process import GaussianProcessRegressor

In [26]:
### Query MySQL #######################################################################

def querySQL (database, query):
    config = {
    'user': 'root',
    'password': 'jim',
    'host': 'localhost',
    'database': database,
    'raise_on_warnings': True,
    }
    cnx = myconnect.connect(**config) 
    resultset = pd.read_sql_query(query,cnx)
    
    return resultset

#######################################################################################

In [27]:
dfDatasetOrder = querySQL('prom_test_data','SELECT  sequence, mean_score, 35boxstart, 10boxstart, ID FROM inbio_prom_lib  WHERE length(sequence)>5)
sampleCount = len(dfDatasetOrder)

DF = pd.DataFrame(index=range(int(sampleCount*(sampleCount-1)/2)), columns=['ID_1','ID_2','sequence_1','sequence_2','rel_rank'])

SyntaxError: EOL while scanning string literal (<ipython-input-27-fe0879943578>, line 1)

In [23]:
for i in range(len(dfDatasetOrder)):
    ID = dfDatasetOrder.iloc[i,4]
    DF.iloc[i*sampleCount:(i+1)*sampleCount,0] = ID

ZIP = list(itertools.combinations(dfDatasetOrder.iloc[:,4],2))

DF['ID_1'] = [item[0] for item in ZIP]
DF['ID_2'] = [item[1] for item in ZIP]

DF['sequence_1'] = [dfDatasetOrder[dfDatasetOrder['ID']==x]['sequence'].values[0] for x in DF['ID_1']]
DF['sequence_2'] = [dfDatasetOrder[dfDatasetOrder['ID']==x]['sequence'].values[0] for x in DF['ID_2']]

DF['score_1'] = [dfDatasetOrder[dfDatasetOrder['ID']==x]['mean_score'].values[0] for x in DF['ID_1']]
DF['score_2'] = [dfDatasetOrder[dfDatasetOrder['ID']==x]['mean_score'].values[0] for x in DF['ID_2']]

DF['rank'] = [1 if x>(DF.iloc[i]['score_2']) else -1 for i, x in enumerate(DF['score_1']) ]

DF['35boxstart_1'] = [dfDatasetOrder[dfDatasetOrder['ID']==x]['35boxstart'].values[0] for x in DF['ID_1']]
DF['35boxstart_2'] = [dfDatasetOrder[dfDatasetOrder['ID']==x]['35boxstart'].values[0] for x in DF['ID_2']]

DF['10boxstart_1'] = [dfDatasetOrder[dfDatasetOrder['ID']==x]['10boxstart'].values[0] for x in DF['ID_1']]
DF['10boxstart_2'] = [dfDatasetOrder[dfDatasetOrder['ID']==x]['10boxstart'].values[0] for x in DF['ID_2']]

In [24]:
DF.to_csv(path_or_buf='/home/jim/Doctoraat/Data/mutalik_library/pw_inbio_prom_lib', header=False,index=False)

738.39999999999998